In [1]:
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sbn
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import  tqdm, trange

In [2]:
pts = gpd.read_file('../../CIAT/data/BEM_2019_Chiapas_Final_Clean.gpkg')
pts['SIM'] = ['CH{0:05d}'.format(i + 1) for i in range(len(pts))]
pts = pts.set_index('SIM')
pts['ref_date'] = pd.to_datetime(pts.fecha_de_siembra)

In [ ]:
yl = []
for i, key in enumerate(tqdm(SIM)):
    crop_start_date = pts.ref_date.dt.date.loc[key[1:]]
    sim_start_date = crop_start_date - dt.timedelta(days = 90)
    crop_end_date = crop_start_date + dt.timedelta(days = 210)
    
    wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE{key}_npf.csv'
    w = pd.read_csv(wfile, skiprows=13)
    w.DAY = pd.to_datetime(w.DAY, format= '%Y%m%d').dt.date
    ini_idx = np.where(w.DAY == sim_start_date)[0][0]
    end_idx = np.where(w.DAY == crop_end_date)[0][0]
    s_idx = slice(ini_idx, end_idx)
    w_seed = w[s_idx].values[:,1:-1].astype('float')
    w_seed = transform(w_seed)
    y_pred = model(w_seed).data.cpu().numpy()
    w_adv = get_adv(w_seed, 0.01)
    y_adv = model(w_adv).data.cpu().numpy()
    w_adv = back_transform(w_adv)
    yl.append([y_pred, y_adv])

    dff = w.iloc[s_idx].copy()
    dff.iloc[:,1:-1] = w_adv
    
    save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE{key}_adv.csv'
    with open(save_file, 'w') as sf:
        with open(wfile) as f:
            for r in range(14):
                sf.writelines(f.readline())
    
    dff.DAY = pd.to_datetime(dff.DAY).dt.strftime('%Y%m%d')
    dff.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)


In [7]:
for i, key in enumerate(tqdm(pts.index)):
    crop_start_date = pts.ref_date.dt.date.loc[key]
    sim_start_date = crop_start_date - dt.timedelta(days = 90)
    crop_end_date = crop_start_date + dt.timedelta(days = 210)
    
    wfile = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_npf.csv'
    w = pd.read_csv(wfile, skiprows=13)
    w.DAY = pd.to_datetime(w.DAY, format= '%Y%m%d').dt.date
    ini_idx = np.where(w.DAY == sim_start_date)[0][0]
    end_idx = np.where(w.DAY == crop_end_date)[0][0]
    s_idx = slice(ini_idx, end_idx)
    dff = w.iloc[s_idx].copy()
    
    save_file = f'/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{key}_npff.csv'
    with open(save_file, 'w') as sf:
        with open(wfile) as f:
            for r in range(14):
                sf.writelines(f.readline())
    
    dff.DAY = pd.to_datetime(dff.DAY).dt.strftime('%Y%m%d')
    dff.to_csv(save_file, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)
